Toms Code zusammengefasst, die Zeiten der beiden Messungen jeweils gemittelt, daraus dann die velocity und viscosity berechnet und in einer csv gespeichert.

In [6]:
import pandas as pd
import numpy as np
import math

# Experiment 1 und 2: Zeitreihen
data_1 = {
    "time_min": [5, 10, 15, 20, 30, 40, 50, 60, 70],
    "t_B": [7.35, 42.38, 23.28, 23.27, 30.57, 28.85, 25.49, 29.1, 31.0],
    "winkel_deg": [30, 30, 45, 45, 60, 75, 75, 75, 75],
    "strecke_cm": [7, 1, 1, 1, 1.6, 1, 1, 1, 1]
}
df1 = pd.DataFrame(data_1)

data_2 = {
    "time_min": [5, 10, 15, 20, 35, 40, 50, 60, 70],
    "t_B": [2.6, 13, 40.7, 23.0, 150, 52, 83, 29.45, 61],
    "winkel_deg": [30, 30, 45, 45, 60, 75, 75, 75, 75],
    "strecke_cm": [7, 1, 1, 1, 1, 1, 1, 1, 1]
}
df2 = pd.DataFrame(data_2)

# Experiment 300µl: Konzentration
data_3 = {
    "Konzentration": [1, 2.5, 4, 5],
    "t_B": [1.48, 4.98, 9.01, 29.1],
    "winkel_deg": [45, 45, 60, 75],
    "strecke_cm": [7, 7, 1, 1]
}
df3 = pd.DataFrame(data_3)

data_4 = {
    "Konzentration": [1, 2.5, 4, 5],
    "t_B": [1.26, 4.71, 11.68, 29.45],
    "winkel_deg": [45, 45, 60, 75],
    "strecke_cm": [7, 7, 1, 1]
}
df4 = pd.DataFrame(data_4)

# Mittelung nur der t_B Spalte für Zeitreihen
merged_time = df1.copy()
merged_time['t_B'] = (df1['t_B'] + df2['t_B']) / 2

# Mittelung nur der t_B Spalte für Konzentration
merged_conc = df3.copy()
merged_conc['t_B'] = (df3['t_B'] + df4['t_B']) / 2

# Berechnung des Kapillardurchmessers
V = 100  # µl = 0.001 cm^3
V = V * 0.001  # cm^3
h = 9  # cm
r_kapillare_cm = math.sqrt(V / (np.pi * h))
d_kapillare_cm = r_kapillare_cm * 2

# Konstanten
K = 0.000583
roh_fluid = 1  # g/cm^3
roh_kugel = 7.85  # g/cm^3
g = 9.81  # m/s^2
d_kugel_mm = 0.6  # mm

# Geschwindigkeit und Viskosität berechnen
def calculate_velocity(strecke_cm, t_B):
    return (strecke_cm * 10) / t_B  # mm/s

def calculate_viscosity(K, roh_fluid, roh_kugel, winkel_deg, d_tube_cm, d_kugel_mm, g, velocity_mm_s):
    d_tube_mm = d_tube_cm * 10
    viscosity = (5 * math.pi / 42 * K * d_kugel_mm**2 * roh_fluid * g * math.sin(math.radians(winkel_deg))) / velocity_mm_s * (roh_kugel - roh_fluid) / roh_fluid * (d_tube_mm + d_kugel_mm) / d_kugel_mm
    return viscosity  # [Pa*s]

# Berechnung für Zeitreihen
merged_time['velocity_mm_s'] = merged_time.apply(lambda row: calculate_velocity(row['strecke_cm'], row['t_B']), axis=1)
merged_time['viscosity_Pa_s'] = merged_time.apply(lambda row: calculate_viscosity(K, roh_fluid, roh_kugel, row['winkel_deg'], d_kapillare_cm, d_kugel_mm, g, row['velocity_mm_s']), axis=1)

# Berechnung für Konzentration
merged_conc['velocity_mm_s'] = merged_conc.apply(lambda row: calculate_velocity(row['strecke_cm'], row['t_B']), axis=1)
merged_conc['viscosity_Pa_s'] = merged_conc.apply(lambda row: calculate_viscosity(K, roh_fluid, roh_kugel, row['winkel_deg'], d_kapillare_cm, d_kugel_mm, g, row['velocity_mm_s']), axis=1)

# Speichern der Ergebnisse in zwei separate CSV-Dateien
merged_time.to_csv('time_results.csv', index=False)
merged_conc.to_csv('conc_results.csv', index=False)

# Ausgabe der Ergebnisse als übersichtliche Tabellen
print("\nMittelwerte Zeitreihen (nur t_B gemittelt):")
print(merged_time)

print("\nMittelwerte Konzentration (nur t_B gemittelt):")
print(merged_conc)



Mittelwerte Zeitreihen (nur t_B gemittelt):
   time_min     t_B  winkel_deg  strecke_cm  velocity_mm_s  viscosity_Pa_s
0         5   4.975          30         7.0      14.070352        0.000559
1        10  27.690          30         1.0       0.361141        0.021780
2        15  31.990          45         1.0       0.312598        0.035584
3        20  23.135          45         1.0       0.432246        0.025735
4        30  90.285          60         1.6       0.177217        0.076875
5        40  40.425          75         1.0       0.247372        0.061426
6        50  54.245          75         1.0       0.184349        0.082426
7        60  29.275          75         1.0       0.341588        0.044484
8        70  46.000          75         1.0       0.217391        0.069898

Mittelwerte Konzentration (nur t_B gemittelt):
   Konzentration     t_B  winkel_deg  strecke_cm  velocity_mm_s  \
0            1.0   1.370          45           7      51.094891   
1            2.5   4.84

time_results.csv ist für die Werte aus 3.1 und conc_results.csv ist für 3.2.1